# Step 1

## Import The Required Functions And Modules

In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download The Source File From The Cloud

In [2]:
#!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip


## Extract The Zip Files

In [3]:
# nzip datasource.zip -d dealership_data

import zipfile
with zipfile.ZipFile('D:\PycharmProjects\GettingStartedWithSQL\datasource.zip', 'r') as zip_ref:
    zip_ref.extractall('D:\PycharmProjects\GettingStartedWithSQL\dealership_data')

## Set The Path For The Target Files

In [4]:
tmpfile    = "dealership_temp.tmp"               # store all extracted data

logfile    = "dealership_logfile.txt"            # all event logs will be stored

targetfile = "dealership_transformed_data.csv"   # transformed data is stored

# Step 2

## EXTRACT

## CSV Extract Function

In [1]:
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe

## JSON Extract Function

In [6]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

## XML Extract Function

In [7]:
def extract_from_xml(file_to_process):

    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])

    tree = ET.parse(file_to_process) 

    root = tree.getroot() 

    for person in root: 

        car_model = person.find("car_model").text 

        year_of_manufacture = int(person.find("year_of_manufacture").text)

        price = float(person.find("price").text) 

        fuel = person.find("fuel").text 

        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True) 

        return dataframe

## Extract Function

In [8]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) 
    #for csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
    #for json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    #for xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
    return extracted_data

## Step 3

## Transform

In [9]:
def transform(data):
       data['price'] = round(data.price, 2)
       return data

## Step 4

## Loading And Logging

## Load Function

In [10]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile, index = False)

## Log Function

In [11]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f: f.write(timestamp + ',' + message + 'n')

## Step 5

## Running The ETL Process

In [12]:
log("ETL Job Started")

In [13]:
log("Extract phase Started")
extracted_data = extract() 
log("Extract phase Ended")

In [14]:
log('Transform phase Started')
transformed_data = transform(extracted_data)
log("Transform phase Ended")

In [15]:
log("Load phase Started")
load(targetfile, transformed_data)
log("Load phase Ended")

In [16]:
log("ETL Job Ended")

## Loading Into The Database

In [17]:
import pandas as pd
empdata = pd.read_csv('D:\\PycharmProjects\\GettingStartedWithSQL\\dealership_transformed_data.csv', index_col=False, delimiter = ',')
empdata.head()

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel


In [3]:
import mysql.connector as msql
from mysql.connector import Error
import csv

In [22]:
try:
    conn = msql.connect(host='localhost', user='root', password='')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE dealership_data")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Database is created


In [23]:
try:
    conn = msql.connect(host='localhost', user='root', password='', database='dealership_data')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE TABLE dealership(car_model varchar(255), year_of_manufacture int(4), price float(30), fuel varchar(255))")
        print("Table is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Table is created


In [6]:
import pandas as pd
dealership_data = pd.read_csv('D:\\PycharmProjects\\GettingStartedWithSQL\\dealership_transformed_data.csv', index_col=False, delimiter = ',')
dealership_data.head()

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel


In [50]:
try:
    conn = msql.connect(host='localhost', user='root', password='', database='dealership_data')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor()
        for i,row in dealership_data.iterrows():
#             print(i, row)
#             print(row)
            print(tuple(row))
            sql = f"INSERT INTO dealership_data.dealership(car_model, year_of_manufacture, price, fuel) VALUES {tuple(row)}"
            print(sql,tuple(row))
            cursor.execute(sql)
            print("Record inserted")
            conn.commit()
except Error as e:
    print("Error while connecting to MySQL", e)

('ritz', 2014, 5000.0, 'Petrol')
INSERT INTO dealership_data.dealership(car_model, year_of_manufacture, price, fuel) VALUES ('ritz', 2014, 5000.0, 'Petrol') ('ritz', 2014, 5000.0, 'Petrol')
Record inserted
('sx4', 2013, 7089.55, 'Diesel')
INSERT INTO dealership_data.dealership(car_model, year_of_manufacture, price, fuel) VALUES ('sx4', 2013, 7089.55, 'Diesel') ('sx4', 2013, 7089.55, 'Diesel')
Record inserted
('ciaz', 2017, 10820.9, 'Petrol')
INSERT INTO dealership_data.dealership(car_model, year_of_manufacture, price, fuel) VALUES ('ciaz', 2017, 10820.9, 'Petrol') ('ciaz', 2017, 10820.9, 'Petrol')
Record inserted
('wagon r', 2011, 4253.73, 'Petrol')
INSERT INTO dealership_data.dealership(car_model, year_of_manufacture, price, fuel) VALUES ('wagon r', 2011, 4253.73, 'Petrol') ('wagon r', 2011, 4253.73, 'Petrol')
Record inserted
('swift', 2014, 6865.67, 'Diesel')
INSERT INTO dealership_data.dealership(car_model, year_of_manufacture, price, fuel) VALUES ('swift', 2014, 6865.67, 'Diesel') 

('omni', 2012, 1865.67, 'Petrol')
INSERT INTO dealership_data.dealership(car_model, year_of_manufacture, price, fuel) VALUES ('omni', 2012, 1865.67, 'Petrol') ('omni', 2012, 1865.67, 'Petrol')
Record inserted
('ciaz', 2014, 11194.03, 'Petrol')
INSERT INTO dealership_data.dealership(car_model, year_of_manufacture, price, fuel) VALUES ('ciaz', 2014, 11194.03, 'Petrol') ('ciaz', 2014, 11194.03, 'Petrol')
Record inserted
('ritz', 2013, 3955.22, 'Petrol')
INSERT INTO dealership_data.dealership(car_model, year_of_manufacture, price, fuel) VALUES ('ritz', 2013, 3955.22, 'Petrol') ('ritz', 2013, 3955.22, 'Petrol')
Record inserted
('wagon r', 2006, 1567.16, 'Petrol')
INSERT INTO dealership_data.dealership(car_model, year_of_manufacture, price, fuel) VALUES ('wagon r', 2006, 1567.16, 'Petrol') ('wagon r', 2006, 1567.16, 'Petrol')
Record inserted
('ertiga', 2015, 8656.72, 'Petrol')
INSERT INTO dealership_data.dealership(car_model, year_of_manufacture, price, fuel) VALUES ('ertiga', 2015, 8656.72,

In [29]:
dealership_data.dtypes


car_model               object
year_of_manufacture      int64
price                  float64
fuel                    object
dtype: object

In [51]:
# Execute query
sql = "SELECT * FROM dealership_data.dealership"
cursor.execute(sql)
# Fetch all the records
result = cursor.fetchall()
for i in result:
    print(i)

('ritz', 2014, 5000.0, 'Petrol')
('sx4', 2013, 7089.55, 'Diesel')
('ciaz', 2017, 10820.9, 'Petrol')
('wagon r', 2011, 4253.73, 'Petrol')
('swift', 2014, 6865.67, 'Diesel')
('vitara brezza', 2018, 13805.97, 'Diesel')
('ciaz', 2015, 10074.63, 'Petrol')
('s cross', 2015, 9701.49, 'Diesel')
('ciaz', 2016, 13059.7, 'Diesel')
('ciaz', 2015, 11119.4, 'Diesel')
('alto 800', 2017, 4253.73, 'Petrol')
('ciaz', 2015, 10223.88, 'Diesel')
('ciaz', 2015, 11194.03, 'Petrol')
('ertiga', 2015, 9104.48, 'Petrol')
('dzire', 2009, 3358.21, 'Petrol')
('ertiga', 2016, 11567.16, 'Diesel')
('ertiga', 2015, 10820.9, 'Diesel')
('ertiga', 2016, 11567.16, 'Diesel')
('wagon r', 2015, 4850.75, 'CNG')
('sx4', 2010, 3955.22, 'Petrol')
('alto k10', 2016, 4253.73, 'Petrol')
('ignis', 2017, 7313.43, 'Petrol')
('sx4', 2011, 6567.16, 'Petrol')
('alto k10', 2014, 3731.34, 'Petrol')
('wagon r', 2013, 4328.36, 'Petrol')
('swift', 2011, 4477.61, 'Petrol')
('swift', 2013, 6194.03, 'Petrol')
('swift', 2017, 8955.22, 'Petrol')
('

In [7]:
try:
    conn = msql.connect(host='localhost', database='dealership_data', user='root', password='')
    if conn.is_connected():
        cursor = conn.cursor()
        for i,row in dealership_data.iterrows():
            #here %S means string values 
            sql = "INSERT INTO dealership_data.dealership VALUES (%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
except Error as e:
            print("Error while connecting to MySQL", e)

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

In [8]:
# Execute query
sql = "SELECT * FROM dealership_data.dealership"
cursor.execute(sql)
# Fetch all the records
result = cursor.fetchall()
for i in result:
    print(i)

('ritz', 2014, 5000.0, 'Petrol')
('sx4', 2013, 7089.55, 'Diesel')
('ciaz', 2017, 10820.9, 'Petrol')
('wagon r', 2011, 4253.73, 'Petrol')
('swift', 2014, 6865.67, 'Diesel')
('vitara brezza', 2018, 13805.97, 'Diesel')
('ciaz', 2015, 10074.63, 'Petrol')
('s cross', 2015, 9701.49, 'Diesel')
('ciaz', 2016, 13059.7, 'Diesel')
('ciaz', 2015, 11119.4, 'Diesel')
('alto 800', 2017, 4253.73, 'Petrol')
('ciaz', 2015, 10223.88, 'Diesel')
('ciaz', 2015, 11194.03, 'Petrol')
('ertiga', 2015, 9104.48, 'Petrol')
('dzire', 2009, 3358.21, 'Petrol')
('ertiga', 2016, 11567.16, 'Diesel')
('ertiga', 2015, 10820.9, 'Diesel')
('ertiga', 2016, 11567.16, 'Diesel')
('wagon r', 2015, 4850.75, 'CNG')
('sx4', 2010, 3955.22, 'Petrol')
('alto k10', 2016, 4253.73, 'Petrol')
('ignis', 2017, 7313.43, 'Petrol')
('sx4', 2011, 6567.16, 'Petrol')
('alto k10', 2014, 3731.34, 'Petrol')
('wagon r', 2013, 4328.36, 'Petrol')
('swift', 2011, 4477.61, 'Petrol')
('swift', 2013, 6194.03, 'Petrol')
('swift', 2017, 8955.22, 'Petrol')
('

In [9]:
try:
    conn = msql.connect(host='localhost', user='root', password='', database='dealership_data')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE TABLE dealerships(car_model varchar(255), year_of_manufacture int(4), price float(30), fuel varchar(255))")
        print("Table is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Table is created


In [10]:
try:
    conn = msql.connect(host='localhost', database='dealership_data', user='root', password='')
    if conn.is_connected():
        cursor = conn.cursor()
        for i,row in dealership_data.iterrows():
            #here %S means string values 
            sql = "INSERT INTO dealership_data.dealerships VALUES (%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
except Error as e:
            print("Error while connecting to MySQL", e)

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i